In [ ]:
!pip install boto3 numpy pandas sagemaker

In [ ]:
 S3 prefix
prefix = "DEMO-scikit-byo-iris"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.serializers import CSVSerializer
role = get_execution_role()

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
WORK_DIRECTORY = "../src/data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-deploy-terraform:latest".format(account, region)

tree = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
)

In [ ]:
tree.fit(data_location)

In [ ]:
predictor = tree.deploy(1, "ml.m4.xlarge", serializer=CSVSerializer)

In [ ]:
shape = pd.read_csv("../src/data/iris.csv", header=None)
shape.sample(3)

In [ ]:
# drop the label column in the training set
shape.drop(shape.columns[[0]], axis=1, inplace=True)
shape.sample(3)

In [ ]:
import itertools

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = shape.iloc[indices[:-1]]

In [ ]:
print(predictor.predict(test_data.values).decode("utf-8"))

In [ ]:
#sess.delete_endpoint(predictor.endpoint)